>>> ### Gpt-3.5-turbo model fine tuning 1st way 

In [69]:
import json,os
import random 
from openai import OpenAI 

In [70]:
# configure gpt client 
gpt_api_key = "past your api key"
client = OpenAI(api_key=gpt_api_key)

In [3]:
# Load your training dataset
with open(r'raw_data\data_conversational_chat_format.json', 'r') as file:
    training_data = json.load(file)
print(training_data)

[{'messages': [{'role': 'system', 'content': 'You are a knowledgeable training assistant.'}, {'role': 'user', 'content': 'What are the key skills needed for a career in Data Science?'}, {'role': 'assistant', 'content': 'Key skills for a career in Data Science include programming (Python, R), statistics, data visualization, machine learning, and knowledge of databases (SQL, NoSQL).'}]}, {'messages': [{'role': 'system', 'content': 'You are an informative training assistant.'}, {'role': 'user', 'content': 'Can you explain what Full Stack Development involves?'}, {'role': 'assistant', 'content': 'Full Stack Development involves working on both the front-end and back-end of web applications. This includes using HTML, CSS, and JavaScript for the front-end, and languages like Node.js, Python, or Ruby for the back-end.'}]}, {'messages': [{'role': 'system', 'content': 'You are a helpful assistant.'}, {'role': 'user', 'content': 'What are some popular tools used in DevOps?'}, {'role': 'assistant

In [4]:
# printing top 5 training examples 
count = 1
for example in training_data:
    print(example)
    count += 1 
    if count == 5:
        break


{'messages': [{'role': 'system', 'content': 'You are a knowledgeable training assistant.'}, {'role': 'user', 'content': 'What are the key skills needed for a career in Data Science?'}, {'role': 'assistant', 'content': 'Key skills for a career in Data Science include programming (Python, R), statistics, data visualization, machine learning, and knowledge of databases (SQL, NoSQL).'}]}
{'messages': [{'role': 'system', 'content': 'You are an informative training assistant.'}, {'role': 'user', 'content': 'Can you explain what Full Stack Development involves?'}, {'role': 'assistant', 'content': 'Full Stack Development involves working on both the front-end and back-end of web applications. This includes using HTML, CSS, and JavaScript for the front-end, and languages like Node.js, Python, or Ruby for the back-end.'}]}
{'messages': [{'role': 'system', 'content': 'You are a helpful assistant.'}, {'role': 'user', 'content': 'What are some popular tools used in DevOps?'}, {'role': 'assistant', 

In [5]:
## Gpt-3.5-turbo required in jsonl format 
print("Type of data object : ",type(training_data))   

Type of data object :  <class 'list'>


In [6]:
## splitting data into training and testing  
testing_size = 0.10
no_of_data_point_in_testing = int(len(training_data)*testing_size)
print("Total no. of data points : ",len(training_data) )

random.shuffle(training_data)   # shuffling training data before select testing 


train_data = training_data[no_of_data_point_in_testing+1:]
val_data = training_data[:no_of_data_point_in_testing]
print("successfully splitted data! ")
print('Training data points : ',len(train_data))
print('Testing data points : ',len(val_data))


Total no. of data points :  71
successfully splitted data! 
Training data points :  63
Testing data points :  7


In [7]:
## writing formated data into memory as a jsonl format  
def save_formatted_data(data,file_path):
    with open(file_path,'w') as file: 
        for example in data:
            json.dump(example,file)
            file.write('\n')



os.makedirs('final_formated_data',exist_ok=True)
training_filepath = r"final_formated_data\train_formated.jsonl"
validation_filepath = r"final_formated_data\val_formated.jsonl"


save_formatted_data(train_data,training_filepath)
save_formatted_data(val_data,validation_filepath)


In [8]:
# to checkout the formatted data
with open(validation_filepath, 'r') as file:
    # Each line in the file is a separate JSON object
    data = [json.loads(line) for line in file]

# Display the examples entry one by one 
for entry in data:
    print(entry)


{'messages': [{'role': 'system', 'content': 'You are a friendly assistant.'}, {'role': 'user', 'content': 'What is the role of an API in web development?'}, {'role': 'assistant', 'content': 'An API (Application Programming Interface) allows different software applications to communicate with each other, enabling functionalities like data exchange and service integration.'}]}
{'messages': [{'role': 'system', 'content': 'You are an informative training assistant.'}, {'role': 'user', 'content': 'Can you explain what RESTful APIs are?'}, {'role': 'assistant', 'content': 'RESTful APIs are application programming interfaces that adhere to the principles of Representational State Transfer (REST). They allow different software systems to communicate over HTTP using standard methods like GET, POST, PUT, and DELETE.'}]}
{'messages': [{'role': 'system', 'content': 'You are an informative training assistant.'}, {'role': 'user', 'content': 'What is a data pipeline?'}, {'role': 'assistant', 'content

In [22]:
# Upload  Training Files
training_file = client.files.create(
    file=open(training_filepath, "rb"), purpose="fine-tune"
)

In [21]:
# Upload  Validation Files
validation_file = client.files.create(
    file=open(validation_filepath, "rb"), purpose="fine-tune"
)

In [23]:
## uploaded files id 
print("Training file id : ",training_file.id )
print("Training file id : ",validation_file.id )


Training file id :  file-rhdu7xF3UzjbII9SOi850ZT8
Training file id :  file-KQxMZGxeyF31Zjwwa0APgnBF


<h2>Fine-tune avaialable models</h2>
<ul>
<li>gpt-4o-2024-08-06</li>
<li>gpt-4o-mini-2024-07-18</li>
<li>gpt-4-0613</li>
<li>gpt-3.5-turbo-0125</li>
<li>gpt-3.5-turbo-1106</li>
<li>gpt-3.5-turbo-0613</li>
<li>babbage-002</li>
<li>davinci-002</li>
</ul>

In [27]:
# creating fine tuning job with model='gpt-3.5-turbo-0125'
sufx_name = "delvex"
response = client.fine_tuning.jobs.create(
  training_file=training_file.id, 
  validation_file=validation_file.id, 
  model="gpt-3.5-turbo",     # or  model='gpt-3.5-turbo-0125'
  suffix= sufx_name
)

In [32]:
## printing fine tune  job id 
print("fine tune job id : ",response.id)

fine tune job id :  ftjob-28M6obAR36MMxxhygdh6ozOb


In [30]:
# List 5 fine-tuning jobs
print(client.fine_tuning.jobs.list(limit=5))
# status== 'runing' 

SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-28M6obAR36MMxxhygdh6ozOb', created_at=1727413987, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-sW9LYXODuBOJUwL7Lo4RNLZu', result_files=[], seed=755621437, status='running', trained_tokens=None, training_file='file-rhdu7xF3UzjbII9SOi850ZT8', validation_file='file-KQxMZGxeyF31Zjwwa0APgnBF', estimated_finish=None, integrations=[], user_provided_suffix='delvex'), FineTuningJob(id='ftjob-RHadzGF6DFLY3rHZPFrbzzNr', created_at=1725623002, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-3.5-turbo-0125:delvex:ashu-walmartbot:A4RkD56G', finished_at=1725623323, hyperparameters=Hyperparameters(n_epochs=5, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organiz

In [36]:
# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(response.id)    # job id  == response.id
## status = 'runing 😥

FineTuningJob(id='ftjob-28M6obAR36MMxxhygdh6ozOb', created_at=1727413987, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-sW9LYXODuBOJUwL7Lo4RNLZu', result_files=[], seed=755621437, status='running', trained_tokens=None, training_file='file-rhdu7xF3UzjbII9SOi850ZT8', validation_file='file-KQxMZGxeyF31Zjwwa0APgnBF', estimated_finish=1727414490, integrations=[], user_provided_suffix='delvex')

In [38]:
client.fine_tuning.jobs.retrieve(response.id)   
# status='succeeded'  <=== it means, Successfully fine-tuned the model  😍 

FineTuningJob(id='ftjob-28M6obAR36MMxxhygdh6ozOb', created_at=1727413987, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-3.5-turbo-0125:delvex:delvex:ABxiEjIy', finished_at=1727414504, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-sW9LYXODuBOJUwL7Lo4RNLZu', result_files=['file-nlAUMhlVPXK7Zk71QJ4vQREN'], seed=755621437, status='succeeded', trained_tokens=10806, training_file='file-rhdu7xF3UzjbII9SOi850ZT8', validation_file='file-KQxMZGxeyF31Zjwwa0APgnBF', estimated_finish=None, integrations=[], user_provided_suffix='delvex')

In [ ]:
# if you want to cancel the job execute it 
# client.fine_tuning.jobs.cancel("paste job id")

In [76]:
# Test the fine-tune model 
def predict(test_messages, fine_tuned_model_id):
    response = client.chat.completions.create(
        model=fine_tuned_model_id, messages=test_messages, temperature=0, max_tokens=250
    )
    return response.choices[0].message.content


In [59]:
## passing the testing point to the finetuned model. 


with open(validation_filepath, 'r') as file:
    test_examples = [json.loads(line) for line in file]

# Display the data
FT_model_id = "ft:gpt-3.5-turbo-0125:delvex:delvex:ABxiEjIy"
for example in test_examples:
    print("User 😎: ",example['messages'][1]['content']) 
    response = predict(test_messages=[example['messages'][1]],fine_tuned_model_id=FT_model_id)
    print("BOT 🤖 : ",response)
    print()

User 😎:  What is the role of an API in web development?
BOT 🤖 :  An API (Application Programming Interface) in web development allows different software programs to communicate with each other, enabling the exchange of data and functionality. It defines the methods and data formats that applications can use to request and exchange information.

User 😎:  Can you explain what RESTful APIs are?
BOT 🤖 :  RESTful APIs are a type of web API that adhere to the principles of REST (Representational State Transfer), using standard HTTP methods (GET, POST, PUT, DELETE) to perform CRUD operations on resources. They are designed to be stateless and use hypermedia links for navigation.

User 😎:  What is a data pipeline?
BOT 🤖 :  A data pipeline is a series of processes that extract, transform, and load data from various sources into a destination, allowing for the automated flow of data.

User 😎:  What is reinforcement learning?
BOT 🤖 :  Reinforcement learning is a type of machine learning where an 

### THANKS TO DELVEX 😊